In [5]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from animal_shelter import AnimalShelter  # Import the CRUD Python module

# Data Manipulation / Model
# Update with your username and password and CRUD Python module name
username = 'aacuser'
password = 'Emerson7'

# Connect to the database via CRUD Module
CRUD = AnimalShelter(username, password, 'nv-desktop-services.apporto.com', 30123)

# Class read method must support return of the list object and accept projection JSON input
# Sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(CRUD.read({}))

# MongoDB v5+ is going to return the '_id' column, and that is going to have an
# invalid object type of 'ObjectID', which will cause the data_table to crash, so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True, it will return a new dataframe that does not contain the dropped column(s).
df.drop(columns=['_id'], inplace=True)

# Dashboard Layout / View
app = JupyterDash('Christine Emerson')

# Place the HTML image tag in the line below into the app.layout code according to your design
# Also remember to include a unique identifier such as your name or date
# html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))
# Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png'  # Grazioso Salvare Logo
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Modify the HTML layout section to include the anchor tag around the image tag
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display': 'none'}),

    # Logo with anchor tag to www.snhu.edu
    html.A(
        html.Img(
            id='customer-image',
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            alt='customer image',
            style={
                'width': '10%',  # Reduce the width to 10% to make it smaller
                'margin': 'auto',  # Centers the image horizontally
                'display': 'block',  # Puts the image on its own line (block element)
            }
        ),
        href='https://www.snhu.edu',  # The URL to the client's home page
        target='_blank'  # Opens the link in a new tab or window
    ),

    # Unique Identifier
    html.Center(html.B(html.H1('Austin Animal Center Outcomes'))),
    html.Center(html.B(html.H3('by Christine Emerson'))),
    html.Hr(),

    # Interactive filtering radio button options.
    html.Div([
        html.B("Filter Animal Type:"),
        html.Br(),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        ),

    ]),
    html.Hr(),
    # DataTable for displaying the animal records
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {'name': i, 'id': i, 'deletable': False, 'selectable': True}
            for i in df.columns
        ],
        data=df.to_dict('records'),

        # Added features to make it more user-friendly for the client
        # Editing the cells
        editable=True,
        # Filtering by column
        filter_action='native',
        # Sorting by column
        sort_action='native',
        # Enable deletion of rows
        row_deletable=True,
        sort_mode='multi',
        # Enable single-row selection for the data table for the mapping callback to function properly.
        row_selectable='single',
        # Selecting columns
        selected_columns=[],
        # Selecting rows
        selected_rows=[0],
        # Paging front-end
        page_action='native',
        # Current page 0
        page_current=0,
        # Animals per page
        page_size=10,
        # Hiding columns
        hidden_columns=[],
    ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',

                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ])
])

# Interaction Between Components / Controller
@app.callback(
    [Output('datatable-id', 'data'), Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Water Rescue filter
    if filter_type == 'water':
        df = pd.DataFrame.from_records(CRUD.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }))
    # Mountain or Wilderness Rescue filter
    elif filter_type == 'mount':
        df = pd.DataFrame.from_records(CRUD.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }))
    # Disaster Rescue or Individual Tracking filter
    elif filter_type == 'disaster':
        df = pd.DataFrame.from_records(CRUD.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }))
    # Reset to no filter
    else:
        df = pd.DataFrame.from_records(CRUD.read({}))

    df.drop(columns=['_id'], inplace=True)  # Drop the _id column from the DataFrame

    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = df.to_dict('records')

    # Returns the output data for the data table
    return data, columns


# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback updates the pie chart with the count of each breed based on the data selected in the data table.
# The viewData argument contains the data available from the data table in the form of a dictionary.
# The pie chart displays the count of each unique breed present in the filtered data. The names list contains
# the breed names, and the values list contains the corresponding counts. The pie chart is updated whenever the
# data in the data table changes, allowing users to visualize the distribution of breeds in the filtered dataset.
# The callback uses Plotly Express to create the pie chart, which is then rendered in the layout of the dashboard.
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # Check if viewData is None or empty
    if viewData is None or not viewData:
        return []

    # Convert the viewData dictionary to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    # Creates the values needed for the names (breed) and values (recurring counts)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    # Create a pie chart based on the data above
    return [
        dcc.Graph(
            figure=px.pie(
                data_frame=dff,
                values=values,
                names=names,
                color_discrete_sequence=px.colors.sequential.RdBu,
                width=800,
                height=500

            )
            .update_traces(textposition='inside')  # Add this line to position the labels inside the pie chart
            .update_layout(uniformtext_minsize=12, uniformtext_mode='hide')  # Add this line for text uniformity
        )
    ]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection, so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)

    # Check if derived_virtual_data is None or empty
    if viewData is None or not viewData:
        return dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id='base-layer-id')
            ]
        )

    if index is None or not index:
        row = 0
    else:
        row = index[0]

    return dl.Map(
        style={'width': '1000px', 'height': '500px'},
        center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id='base-layer-id'),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                      children=[
                          dl.Tooltip(dff.iloc[row, 4]),
                          dl.Popup([
                              html.H1('Animal Name'),
                              html.P(dff.iloc[row, 9])
                          ])
                      ])
        ]
    )


app.run_server(debug=True)


Dash app running on http://127.0.0.1:19981/
